Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic

In [65]:
batch_size = 256

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # First layer computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  # Larger beta -> worse performance
  # Increasing the batch_size value also has an effect
  loss = loss + 5e-8 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 13.434826
Minibatch accuracy: 11.7%
Validation accuracy: 13.3%
Minibatch loss at step 500: 1.535421
Minibatch accuracy: 74.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 0.972226
Minibatch accuracy: 78.9%
Validation accuracy: 76.5%
Minibatch loss at step 1500: 1.006060
Minibatch accuracy: 77.3%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.980643
Minibatch accuracy: 80.1%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 1.172011
Minibatch accuracy: 76.6%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 0.956096
Minibatch accuracy: 78.5%
Validation accuracy: 78.8%
Test accuracy: 86.2%


### NN

In [9]:
batch_size = 256
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
  
  # First layer computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  hidden_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))

  # Second layer computation
  hidden = tf.matmul(tf.nn.relu(logits), hidden_weights) + hidden_biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hidden, tf_train_labels))
  loss = loss + 5e-3 * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(logits), hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), hidden_weights) + hidden_biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1905.680420
Minibatch accuracy: 10.5%
Validation accuracy: 31.4%
Minibatch loss at step 500: 126.420387
Minibatch accuracy: 85.2%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 10.811633
Minibatch accuracy: 89.8%
Validation accuracy: 86.0%
Minibatch loss at step 1500: 1.476457
Minibatch accuracy: 86.3%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.656725
Minibatch accuracy: 87.9%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 0.695659
Minibatch accuracy: 84.4%
Validation accuracy: 86.2%
Minibatch loss at step 3000: 0.583184
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Test accuracy: 92.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

Answer: test and validation accuracy drops.

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

Answer: performance on test and validation set drops to around 10%.

---

In [19]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
  biases = tf.Variable(tf.zeros([hidden_size]))
  
  # First layer computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  hidden_weights = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))

  # Second layer computation
  hidden = tf.matmul(tf.nn.relu(tf.nn.dropout(logits, 0.5)), hidden_weights) + hidden_biases

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hidden, tf_train_labels))
  loss = loss + 5e-4 * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(logits), hidden_weights) + hidden_biases)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases), hidden_weights) + hidden_biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases), hidden_weights) + hidden_biases)

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 495.987976
Minibatch accuracy: 6.2%
Validation accuracy: 29.8%
Minibatch loss at step 500: 27.876141
Minibatch accuracy: 77.3%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 10.593653
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 20.774580
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 17.538538
Minibatch accuracy: 74.2%
Validation accuracy: 78.0%
Minibatch loss at step 2500: 13.230665
Minibatch accuracy: 78.9%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 12.409760
Minibatch accuracy: 82.0%
Validation accuracy: 80.1%
Test accuracy: 86.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [9]:
batch_size = 2000
layer_size = [image_size * image_size, 1024, 384, 384, 384, num_labels]
stddev = 0.05

graph = tf.Graph()
with graph.as_default():

  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([layer_size[0], layer_size[1]], stddev=stddev))
  biases = tf.Variable(tf.zeros([layer_size[1]]))
  
  # Input layer computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  logits = tf.nn.relu(logits)

  # Hidden and output layer computations
  layer_1_weights = tf.Variable(tf.truncated_normal([layer_size[1], layer_size[2]], stddev=stddev))
  layer_1_biases = tf.Variable(tf.zeros([layer_size[2]]))
  layer_1 = tf.matmul(logits, layer_1_weights) + layer_1_biases
  layer_1 = tf.nn.dropout(layer_1, 0.5)
  layer_1 = tf.nn.relu(layer_1)

  layer_2_weights = tf.Variable(tf.truncated_normal([layer_size[2], layer_size[3]], stddev=stddev))
  layer_2_biases = tf.Variable(tf.zeros([layer_size[3]]))
  layer_2 = tf.matmul(layer_1, layer_2_weights) + layer_2_biases
  layer_2 = tf.nn.dropout(layer_2, 0.5)
  layer_2 = tf.nn.relu(layer_2)
  
  layer_3_weights = tf.Variable(tf.truncated_normal([layer_size[3], layer_size[4]], stddev=stddev))
  layer_3_biases = tf.Variable(tf.zeros([layer_size[4]]))
  layer_3 = tf.matmul(layer_2, layer_3_weights) + layer_3_biases
  layer_3 = tf.nn.dropout(layer_3, 0.5)
  layer_3 = tf.nn.relu(layer_3)

  layer_4_weights = tf.Variable(tf.truncated_normal([layer_size[4], layer_size[5]], stddev=stddev))
  layer_4_biases = tf.Variable(tf.zeros([layer_size[5]]))
  layer_4 = tf.matmul(layer_3, layer_4_weights) + layer_4_biases
  layer_4 = tf.nn.dropout(layer_4, 0.5)
  layer_4 = tf.nn.relu(layer_4)

  output_layer = layer_4

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer, tf_train_labels))
  #loss = loss + 5e-4 * (tf.nn.l2_loss(weights) + 
  #                      tf.nn.l2_loss(layer_1_weights) + 
  #                     tf.nn.l2_loss(layer_2_weights) + 
  #                      tf.nn.l2_loss(layer_3_weights))
  
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  #learning_rate = tf.train.exponential_decay(0.5, global_step, 50, 0.99, staircase=True)
  learning_rate = 0.5
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  def predict(dataset):
    input_layer = tf.matmul(dataset, weights) + biases
    input_layer = tf.nn.relu(input_layer)
    layer_1 = tf.matmul(input_layer, layer_1_weights) + layer_1_biases
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.matmul(layer_1, layer_2_weights) + layer_2_biases
    layer_2 = tf.nn.relu(layer_2)
    layer_3 = tf.matmul(layer_2, layer_3_weights) + layer_3_biases
    layer_3 = tf.nn.relu(layer_3)
    layer_4 = tf.matmul(layer_3, layer_4_weights) + layer_4_biases
    layer_4 = tf.nn.relu(layer_4)
    output_layer = layer_4
    return tf.nn.softmax(output_layer)

  # Predictions for the training, validation, and test data.
  train_prediction = predict(tf_train_dataset)
  valid_prediction = predict(tf_valid_dataset)
  test_prediction = predict(tf_test_dataset)

num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.387470
Minibatch accuracy: 7.5%
Validation accuracy: 14.3%
Minibatch loss at step 100: 1.548380
Minibatch accuracy: 82.1%
Validation accuracy: 83.0%
Minibatch loss at step 200: 1.447778
Minibatch accuracy: 84.2%
Validation accuracy: 84.7%
Minibatch loss at step 300: 1.460342
Minibatch accuracy: 84.7%
Validation accuracy: 85.6%
Minibatch loss at step 400: 1.435648
Minibatch accuracy: 86.2%
Validation accuracy: 85.9%
Minibatch loss at step 500: 1.418079
Minibatch accuracy: 86.0%
Validation accuracy: 86.4%
Minibatch loss at step 600: 1.433276
Minibatch accuracy: 86.9%
Validation accuracy: 86.6%
Minibatch loss at step 700: 1.379535
Minibatch accuracy: 88.4%
Validation accuracy: 87.2%
Minibatch loss at step 800: 1.384121
Minibatch accuracy: 87.0%
Validation accuracy: 87.5%
Minibatch loss at step 900: 1.444559
Minibatch accuracy: 87.2%
Validation accuracy: 87.7%
Minibatch loss at step 1000: 1.370703
Minibatch accuracy: 88.0%
Validation accuracy: 87.9%
